In [25]:
import glob
import pandas as pd

daumNews_dir = glob.glob('./daum_news_dataset/*.csv')
daumNews_dir

['./daum_news_dataset\\daum_news_20210126-20210130.csv',
 './daum_news_dataset\\daum_news_20210129-20210129.csv',
 './daum_news_dataset\\daum_news_20210130-20210130.csv']

In [26]:
import glob
import pandas as pd
import re
import os
import sqlite3
import numpy as np


def preprocessing_articles(data):
    
    contents_new = []   
    for content in data['content']:
        content = str(content).split('\n')
        content = list(filter(lambda s: len(s) > 3, content))
        content_prep = content[:-1]
        content_prep = list(filter(lambda s: len(s) > 30 and \
                                             '▶' not in s and \
                                             '©' not in s and \
                                             '▲' not in s and \
                                             '사진=' not in s and \
                                             '사진제공=' not in s and \
                                             '@' not in s and \
                                             not re.findall("기자 *$", s) and \
                                             not re.findall("제공 *$", s) and \
                                             not re.findall("이하", s) and \
                                             not re.findall("무단전재", s) and \
                                             not re.findall("기자 *= *", s), content_prep))

        content_prep = list(map(lambda s: re.sub("\[.*\]", "", s), content_prep))  # [스포츠서울]
        content_prep = list(map(lambda s: re.sub("\(.*\=.*\)", "", s), content_prep))  # (서울=뉴시스)
        content_prep = list(map(lambda s: re.sub("\【.*\=.*\】", "", s), content_prep))  # 【파이낸셜뉴스 포천=강근주 기자】
        content_prep = list(map(lambda s: re.sub('[^가-힣]',' ', s), content_prep)) # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거

        contents_new.append(''.join(content_prep))

    data['news_content'] = contents_new

    return data


if __name__ == "__main__":
    con = sqlite3.connect("glob_test9.db")
    c = con.cursor()
    c.execute('DROP TABLE IF EXISTS `news_dummy`;')
    c.execute('''
            CREATE TABLE `news_dummy` (
                `news_id` INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                `news_name` TEXT NOT NULL,
                `pubdate` TEXT NOT NULL,
                `url` TEXT NOT NULL,
                `news_content` TEXT NOT NULL
            );

    ''')
       
    for daumNews in daumNews_dir:
        data_prep = pd.DataFrame()
        df = pd.read_csv(daumNews, names=['order', 'news_name', 'pubdate', 'url', 'content'])
        df = df.drop(['order'], axis=1)
        data_prep = preprocessing_articles(df)
        data_prep['news_content'] = data_prep['news_content'].str.strip()
        data_prep['news_content'] = data_prep['news_content'].replace('', np.nan)
        data_prep_drop = data_prep.dropna(axis=0)
        data_prep_drop_dup = data_prep_drop.drop_duplicates(['news_content'], keep='first', inplace=False, ignore_index=True)
        result = data_prep_drop_dup[['news_name', 'pubdate', 'url', 'news_content']]
        print(result)
        result.to_sql('news_dummy', con, if_exists='append', index=False)

    con.commit()
    con.close()

                                               news_name              pubdate  \
0      '불청' 새 친구 최제우, 새해맞이 '빙고 게임'으로 웃음 만발..방구석 신혼여행까...  2021. 01. 26. 23:59   
1                IMF, 올 세계성장률 전망치 5.5%로 소폭 상향..작년은 -3.5%  2021. 01. 26. 23:58   
2                          흥국생명 4라운드 전승, "승리의 V자" 한 선수들.  2021. 01. 26. 23:57   
3           '아맛' 함소원, 진화 주식 투자 갈등..성민♥김사은 합류 예고[★밤TView]  2021. 01. 26. 23:57   
4                      프랑스 퐁피두센터 2023년부터 리모델링..2027년 재개장  2021. 01. 26. 23:57   
...                                                  ...                  ...   
95814                '양준혁♥' 박현선, 2번 미룬 결혼식에 "나야? 축의금이야?"  2021. 01. 30. 00:00   
95815            김종국, 립싱크 무대 후 논란 기사.."한동안 쉬었다" ('너목보8')  2021. 01. 30. 00:00   
95816      이서진, 8년 경력 아궁이 불때기..박서준 "스페인서도 곰탕" (윤스테이)[종합]  2021. 01. 30. 00:00   
95817         中 3천억원 '뇌물왕' 형확정 8일만에 사형.. 시진핑 '반부패' 강조 직후  2021. 01. 30. 00:00   
95818               검사 인사 착수한 박범계..이성윤 등 '추미애 라인' 어떻게 되나  2021. 01. 30. 00:00   

                           